# Implementing and understanding the capgen-vid code, mostly the code of the decoder where the LSTM is connected to generate descriptions

### Importing the required libraries

In [30]:
import pickle as pkl
import numpy as np

import os, sys, socket, shutil
import time

from config import config
from jobman import DD, expand
import common
import model_attention

import theano
import theano.tensor as tensor
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams


import copy
import warnings
from collections import OrderedDict

from sklearn.model_selection import KFold
from scipy import optimize, stats

import data_engine
import metrics
import common


from common import *

### Reading the CAP.pkl file

In [14]:
objects = []
with(open("youtube2text_iccv15/worddict.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe0 in position 0: ordinal not in range(128)

In [9]:
objects

[['vid117_17',
  'vid523_28',
  'vid262_27',
  'vid57_47',
  'vid1101_20',
  'vid941_17',
  'vid255_4',
  'vid200_19',
  'vid838_7',
  'vid914_12',
  'vid854_42',
  'vid163_8',
  'vid325_40',
  'vid487_26',
  'vid1042_28',
  'vid528_25',
  'vid689_17',
  'vid675_27',
  'vid1075_3',
  'vid576_9',
  'vid17_28',
  'vid607_8',
  'vid27_36',
  'vid1014_49',
  'vid140_23',
  'vid15_36',
  'vid239_36',
  'vid334_18',
  'vid210_21',
  'vid843_33',
  'vid709_5',
  'vid898_5',
  'vid569_42',
  'vid462_15',
  'vid769_22',
  'vid620_25',
  'vid929_26',
  'vid183_6',
  'vid1168_29',
  'vid738_30',
  'vid415_29',
  'vid932_10',
  'vid664_4',
  'vid340_21',
  'vid471_11',
  'vid983_2',
  'vid868_9',
  'vid808_25',
  'vid254_51',
  'vid943_22',
  'vid989_26',
  'vid1027_20',
  'vid757_41',
  'vid405_20',
  'vid104_24',
  'vid104_14',
  'vid1182_26',
  'vid678_37',
  'vid254_42',
  'vid799_34',
  'vid674_0',
  'vid1069_40',
  'vid938_0',
  'vid606_23',
  'vid678_55',
  'vid725_12',
  'vid186_23',
  'vi

In [5]:
def load_pickle(pickle_file):
    try:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f)
    except UnicodeDecodeError as e:
        with open(pickle_file, 'rb') as f:
            pickle_data = pickle.load(f, encoding='latin1')
    except Exception as e:
        print('Unable to load data ', pickle_file, ':', e)
        raise
    return pickle_data

In [16]:
objects = load_pickle("youtube2text_iccv15/worddict.pkl")

In [18]:
word_idict = dict()

In [19]:
for kk, vv in objects.items():
    word_idict[vv] = kk

In [21]:
word_idict

{2: 'a',
 3: 'is',
 4: 'the',
 5: 'man',
 6: 'woman',
 7: 'on',
 8: 'in',
 9: 'playing',
 10: 'are',
 11: 'of',
 12: 'and',
 13: 'with',
 14: 'person',
 15: 'to',
 16: 'into',
 17: 'an',
 18: 'two',
 19: 'dog',
 20: 'cat',
 21: 'girl',
 22: 'his',
 23: 'boy',
 24: 'cutting',
 25: 'someone',
 26: 'some',
 27: 'riding',
 28: 'dancing',
 29: 'baby',
 30: 'water',
 31: 'guitar',
 32: 'slicing',
 33: 'men',
 34: 'from',
 35: 'lady',
 36: 'walking',
 37: 'eating',
 38: 'up',
 39: 'down',
 40: 'horse',
 41: 'her',
 42: 'car',
 43: 'cooking',
 44: 'doing',
 45: 'running',
 46: 'at',
 47: 'women',
 48: 'people',
 49: 'something',
 50: 'bowl',
 51: 'food',
 52: 'making',
 53: 'talking',
 54: 'out',
 55: 'ball',
 56: 'small',
 57: 'off',
 58: "'s",
 59: 'singing',
 60: 'it',
 61: 'meat',
 62: 'onion',
 63: 'pan',
 64: 'knife',
 65: 'by',
 66: 'potato',
 67: 'one',
 68: 'being',
 69: 'over',
 70: 'young',
 71: 'little',
 72: 'piano',
 73: 'putting',
 74: 'plays',
 75: 'through',
 76: 'how',
 77: '

In [23]:
f = open("youtube2text_iccv15/train.pkl", 'rb')

In [24]:
train = pickle.load(f)

In [26]:
len(train)

48780

In [27]:
path = "youtube2text_iccv15/valid.pkl"

In [28]:
f = open(path, 'rb')

In [29]:
valid = pickle.load(f)

### Config files

In [4]:
config = DD{'model': 'attention', 'random_seed': 1234, 'erase_history': True, 'attention': DD{'reload_': False, 
                                                                                              'save_model_dir': 'Experiments/arctic-capgen-vid/test_non/', 
                                                                                              'from_dir': '', 'dataset': 'youtube2text', 
                                                                                              'video_feature': 'googlenet', 'dim_word': 468, 'ctx_dim': -1, 
                                                                                              'dim': 3518, 'n_layers_out': 1, 'n_layers_init': 0, 
                                                                                              'encoder_dim': 300, 'prev2out': True, 'ctx2out': True, 'patience': 20, 'max_epochs': 500, 'decay_c': 0.0001, 'alpha_entropy_r': 0.0, 'alpha_c': 0.70602, 'lrate': 0.0001, 'selector': True, 'n_words': 20000, 'maxlen': 30, 'optimizer': 'adadelta', 'clip_c': 10.0, 'batch_size': 64, 'valid_batch_size': 200, 'dispFreq': 10, 'validFreq': 2000, 'saveFreq': -1, 'sampleFreq': 100, 'metric': 'everything', 'use_dropout': True, 'K': 28, 'OutOf': None, 'verbose': True, 'debug': False}}

SyntaxError: invalid syntax (<ipython-input-4-826b1c42685b>, line 1)

In [5]:
    args = {}

In [7]:
state = expand(args)

In [8]:
state

DD{}

### Set up the configuration for training files

### Understanding train_model function

In [9]:
def main(state, channel=None):
    set_config(config, state)
    train_from_scratch(config, state, channel)

In [10]:
def set_config(conf, args, add_new_key=False):
    # add_new_key: if conf does not contain the key, creates it
    for key in args:
        if key != 'jobman':
            v = args[key]
            if isinstance(v, DD):
                set_config(conf[key], v)
            else:
                if conf.has_key(key):
                    conf[key] = convert_from_string(v)
                elif add_new_key:
                    # create a new key in conf
                    conf[key] = convert_from_string(v)
                else:
                    raise KeyError(key)

In [11]:
def train_from_scratch(config, state, channel):
    # Model options
    save_model_dir = config[config.model].save_model_dir
    if save_model_dir == 'current':
        config[config.model].save_model_dir = './'
        save_model_dir = './'
        # to facilitate the use of cluster for multiple jobs
        save_path = './model_config.pkl'
    else:
        # run locally, save locally
        save_path = save_model_dir + 'model_config.pkl'
    print('current save dir ',save_model_dir)
    common.create_dir_if_not_exist(save_model_dir)

    reload_ = config[config.model].reload_
    if reload_:
        print('preparing reload')
        save_dir_backup = config[config.model].save_model_dir
        from_dir_backup = config[config.model].from_dir
        # never start retrain in the same folder
        assert save_dir_backup != from_dir_backup
        print('save dir ',save_dir_backup)
        print('from_dir ',from_dir_backup)
        print('setting current model config with the old one')
        model_config_old = common.load_pkl(from_dir_backup+'/model_config.pkl')
        set_config(config, model_config_old)
        config[config.model].save_model_dir = save_dir_backup
        config[config.model].from_dir = from_dir_backup
        config[config.model].reload_ = True
    if config.erase_history:
        print('erasing everything in ',save_model_dir)
        os.system('rm %s/*'%save_model_dir)
    # for stdout file logging
    #sys.stdout = Unbuffered(sys.stdout, state.save_model_path + 'stdout.log')
    print('saving model config into %s'%save_path)
    common.dump_pkl(config, save_path)
    # Also copy back from config into state.
    for key in config:
        setattr(state, key, config[key])
    model_type = config.model
    print('Model Type: %s'%model_type)
    print('Host:    %s' % socket.gethostname())
    print('Command: %s' % ' '.join(sys.argv))
    if config.model == 'attention':
        model_attention.train_from_scratch(state, channel)
    else:
        raise NotImplementedError()

In [14]:
set_config(config, state)

In [15]:
config

DD{'model': 'attention', 'random_seed': 1234, 'erase_history': True, 'attention': DD{'reload_': False, 'save_model_dir': 'Experiments/arctic-capgen-vid/test_non/', 'from_dir': '', 'dataset': 'youtube2text', 'video_feature': 'googlenet', 'dim_word': 468, 'ctx_dim': -1, 'dim': 3518, 'n_layers_out': 1, 'n_layers_init': 0, 'encoder_dim': 300, 'prev2out': True, 'ctx2out': True, 'patience': 20, 'max_epochs': 500, 'decay_c': 0.0001, 'alpha_entropy_r': 0.0, 'alpha_c': 0.70602, 'lrate': 0.0001, 'selector': True, 'n_words': 20000, 'maxlen': 30, 'optimizer': 'adadelta', 'clip_c': 10.0, 'batch_size': 64, 'valid_batch_size': 200, 'dispFreq': 10, 'validFreq': 2000, 'saveFreq': -1, 'sampleFreq': 100, 'metric': 'everything', 'use_dropout': True, 'K': 28, 'OutOf': None, 'verbose': True, 'debug': False}}

In [16]:
state

DD{}

In [18]:
train_from_scratch(config=config, state=state, channel=None)

current save dir  Experiments/arctic-capgen-vid/test_non/
Experiments/arctic-capgen-vid/test_non/ already exists!
erasing everything in  Experiments/arctic-capgen-vid/test_non/
saving model config into Experiments/arctic-capgen-vid/test_non/model_config.pkl
Model Type: attention
Host:    groot-groot
Command: /home/himansh/env/lib/python3.7/site-packages/ipykernel_launcher.py -f /run/user/1005/jupyter/kernel-c02e743c-38d6-44de-9250-788960e4ed09.json
training an attention model
Loading data
loading youtube2text googlenet features


/media/Seagate_4TB1/himansh/Anirudh/arctic-capgen-vid/model_attention.py:37: UserWarning: Feeding context to output directly seems to hurt.
  warnings.warn('Feeding context to output directly seems to hurt.')


uneven minibath chunking, overall 64, last one 12
uneven minibath chunking, overall 200, last one 91
uneven minibath chunking, overall 200, last one 168
init params
no lstm on ctx


KeyboardInterrupt: 

In [19]:
    save_model_dir = config[config.model].save_model_dir
    if save_model_dir == 'current':
        config[config.model].save_model_dir = './'
        save_model_dir = './'
        # to facilitate the use of cluster for multiple jobs
        save_path = './model_config.pkl'
    else:
        # run locally, save locally
        save_path = save_model_dir + 'model_config.pkl'

In [20]:
save_model_dir

'Experiments/arctic-capgen-vid/test_non/'

In [21]:
    reload_ = config[config.model].reload_
    if reload_:
        print('preparing reload')
        save_dir_backup = config[config.model].save_model_dir
        from_dir_backup = config[config.model].from_dir
        # never start retrain in the same folder
        assert save_dir_backup != from_dir_backup
        print('save dir ',save_dir_backup)
        print('from_dir ',from_dir_backup)
        print('setting current model config with the old one')
        model_config_old = common.load_pkl(from_dir_backup+'/model_config.pkl')
        set_config(config, model_config_old)
        config[config.model].save_model_dir = save_dir_backup
        config[config.model].from_dir = from_dir_backup
        config[config.model].reload_ = True

In [22]:
reload_

False

In [23]:
    if config.erase_history:
        print('erasing everything in ',save_model_dir)
        os.system('rm %s/*'%save_model_dir)

erasing everything in  Experiments/arctic-capgen-vid/test_non/


In [24]:
    for key in config:
        setattr(state, key, config[key])


In [25]:
state

DD{'model': 'attention', 'random_seed': 1234, 'erase_history': True, 'attention': DD{'reload_': False, 'save_model_dir': 'Experiments/arctic-capgen-vid/test_non/', 'from_dir': '', 'dataset': 'youtube2text', 'video_feature': 'googlenet', 'dim_word': 468, 'ctx_dim': -1, 'dim': 3518, 'n_layers_out': 1, 'n_layers_init': 0, 'encoder_dim': 300, 'prev2out': True, 'ctx2out': True, 'patience': 20, 'max_epochs': 500, 'decay_c': 0.0001, 'alpha_entropy_r': 0.0, 'alpha_c': 0.70602, 'lrate': 0.0001, 'selector': True, 'n_words': 20000, 'maxlen': 30, 'optimizer': 'adadelta', 'clip_c': 10.0, 'batch_size': 64, 'valid_batch_size': 200, 'dispFreq': 10, 'validFreq': 2000, 'saveFreq': -1, 'sampleFreq': 100, 'metric': 'everything', 'use_dropout': True, 'K': 28, 'OutOf': None, 'verbose': True, 'debug': False}}

In [26]:
    model_type = config.model
    print(model_type)

attention


### Implementing train_from_scratch from model_attention

In [27]:
base_path = None
hostname = socket.gethostname()
lscratch_dir = None

In [29]:
    t0 = time.time()


### Attention class

In [31]:
class Attention(object):
    
    def __init__(self, channel = None):
        # These are the different functions that perform the initialization of parameters of their respective layers
        self.layers = {
            'ff': ('self.param_init_ff_layer', 'self.fflayer'),
            'lstm': ('self.param_init_lstm', 'self.lstm_layer'),
            'lstm_cond': ('self.param_init_lstm_cond', 'self.lstm_cond_layer'),
        }
        self.channel = channel
        
    def get_layer(self, name):
        """
        Part of the reason the init is very slow is because,
        the layer's constructor is called even when it isn't needed
        """
        fns = self.layers[name]
        return eval((fns[0]), eval(fns[1]))
        
        
    def load_params(self, path, params):
        # load params from disk
        pp = numpy.load(path)
        for kk, vv in params.items():
            if kk not in pp:
                raise Warning('%s is not in the archive'%kk)
            params[kk] = pp[kk]
            
        return params
    
    def init_tparams(self, params, force_cpu=False):
        # initialize Theano shared variables according to the initial parameters
        tparams = OrderedDict()
        for kk, pp in params.items():
            if force_cpu:
                tparams[kk] = theano.tensor._shared(params[kk], name=kk)
            else:
                tparams[kk] = theano.shared(params[kk], name=kk)
        return tparams
    
    def param_init_fflayer(self, options, params, prefix = 'ff', nin = None, nout = None):
        if nin == None:
            nin = options['dim_proj']
        if nout == None:
            nout = options['dim_proj']
            
        params[_p(prefix, 'W')] = norm_weight(nin, nout, scale=0.01)
        params[_p(prefix, 'b')] = np.zeros((nout,)).astype('float32')
        
        return params
    
    def fflayer(self, tparams, state_below, options, prefix='rconv', activ='lambda x: tensor.tanh(x)', **kwargs):
        return eval(activ)(tensor.dot(state_below, tparams[_p(prefix,'W')])+tparams[_p(prefix,'b')])
    
    # LSTM layer
    def param_init_lstm(self, options, params, prefix = None, nin= None, dim = None):
        assert prefix is not None
        
        if nin == None:
            nin = options['dim_proj']
        
        if dim == None:
            dim = options['dim_proj']
            
        # Stack the weight matricies for faster dot prods
        # norm_weight initializes the weights of the matrix by random values
        W = np.concatenate([norm_weight(nin, dim), norm_weight(nin=nin,dim), norm_weight(nin=nin, dim=dim),
                           norm_weight(nin=nin, dim=dim)], axis = 1)
        
        params[_p(prefix,'W')] = W
                
        U = numpy.concatenate([ortho_weight(dim),
                               ortho_weight(dim),
                               ortho_weight(dim),
                               ortho_weight(dim)], axis=1)
        params[_p(prefix,'U')] = U
        params[_p(prefix,'b')] = numpy.zeros((4 * dim,)).astype('float32')
        
        return params
    
    # Implementation of lstm fprop
    def lstm_layer(self, tparams, state_below, options, prefix='lstm', mask = None, forget = False, use_noise = None,
                  trng = None, **kwargs):
        
        nsteps = state_below.shape[0]
        dim = tparams[_p(prefix, 'U')].shape[0]
        
        if state_below.ndim == 3:
            n_samples = state_below.shape[1]
            init_state = tensor.alloc(0., n_samples, dim)
            init_memory = tensor.alloc(0., n_samples, dim)
        else:
            n_samples = 1
            init_state = tensor.alloc(0., dim)
            init_memory = tensor.alloc(0., dim)
            
        if mask == None:
            mask = tensor.alloc(1., state_below.shape[0], 1)
            
        def _slice(_x, n, dim):
            if _x.ndim == 3:
                return _x[:, :, n*dim:(n+1)*dim]
            elif _x.ndim == 2:
                return _x[:, n*dim:(n+1)*dim]
            return _x[n*dim:(n+1)*dim]
        
        def _step(m_, x_, h_, c_, U, b):
            preact = tensor.dot(h_, U)
            preact += x_
            preact += b

            i = tensor.nnet.sigmoid(_slice(preact, 0, dim))
            f = tensor.nnet.sigmoid(_slice(preact, 1, dim))
            o = tensor.nnet.sigmoid(_slice(preact, 2, dim))
            c = tensor.tanh(_slice(preact, 3, dim))

            if forget:
                f = T.zeros_like(f)
            c = f * c_ + i * c
            h = o * tensor.tanh(c)
            if m_.ndim == 0:
                # when using this for minibatchsize=1
                h = m_ * h + (1. - m_) * h_
                c = m_ * c + (1. - m_) * c_
            else:
                h = m_[:,None] * h + (1. - m_)[:,None] * h_
                c = m_[:,None] * c + (1. - m_)[:,None] * c_
            return h, c, i, f, o, preact

        state_below = tensor.dot(state_below, tparams[_p(prefix, 'W')]) + tparams[_p(prefix, 'b')]
        U = tparams[_p(prefix, 'U')]
        b = tparams[_p(prefix, 'b')]
        rval, updates = theano.scan(
            _step,
            sequences=[mask, state_below],
            non_sequences=[U,b],
            outputs_info = [init_state, init_memory, None, None, None, None],
            name=_p(prefix, '_layers'),
            n_steps=nsteps,
            strict=True,
            profile=False)
        return rval
    
    # Conditional LSTM layer with Attention
    def param_init_lstm_cond(self, options, params, prefix='lstm_cond', nin=None, dim=None, dimctx = None):
        if nin == None:
            nin = options['dim']
        if dim == None:
            dim = options['dim']
        if dimctx == None:
            dimctx = options['dim']
            
        # input to LSTM
        W = numpy.concatenate([norm_weight(nin,dim),
                               norm_weight(nin,dim),
                               norm_weight(nin,dim),
                               norm_weight(nin,dim)], axis=1)
        params[_p(prefix,'W')] = W

        # LSTM to LSTM
        U = numpy.concatenate([ortho_weight(dim),
                               ortho_weight(dim),
                               ortho_weight(dim),
                               ortho_weight(dim)], axis=1)
        params[_p(prefix,'U')] = U
        
        # Bias to LSTM
        params[_p(prefix, 'b')] = np.zeros((4 * dim,)).astype('float32')
        
        # Context to LSTM
        Wc = norm_weight(dimctx, dim*4)
        params[_p(prefix, 'Wc')] = Wc
        
        # attention: context -> hidden
        Wc_att = norm_weight(dimctx, ortho=False)
        params[_p(prefix,'Wc_att')] = Wc_att

        # attention: LSTM -> hidden
        Wd_att = norm_weight(dim,dimctx)
        params[_p(prefix,'Wd_att')] = Wd_att

        # attention: hidden bias
        b_att = numpy.zeros((dimctx,)).astype('float32')
        params[_p(prefix,'b_att')] = b_att
        
        # attention:
        U_att = norm_weight(dimctx,1)
        params[_p(prefix,'U_att')] = U_att
        c_att = numpy.zeros((1,)).astype('float32')
        params[_p(prefix, 'c_tt')] = c_att

        if options['selector']:
            # attention: selector
            W_sel = norm_weight(dim, 1)
            params[_p(prefix, 'W_sel')] = W_sel
            b_sel = numpy.float32(0.)
            params[_p(prefix, 'b_sel')] = b_sel

        return params
    
    def lstm_cond_layer(self, tparams, state_below, options, prefix = 'lstm', mask= None, context = None, 
                       one_step = False, init_memory = None, init_state = None, trng = None, use_noise = None, 
                       mode = None, **kwargs):
        
        # state_below (t, m, dim_word), or (m, dim_word) in sampling
        # mask (t, m)
        # context (m, f, dim_ctx), or (f, dim_word) in sampling
        # init_memory, init_state (m, dim)
        assert context, 'Context must be provided'

        if one_step:
            
            assert init_memory, 'previous memory must be provided'
            assert init_state, 'previous state must be provided'
    
        nsteps = state_below.shape[0]
        
        if state_below.ndim == 3:
            n_samples = state_below.shape[1]
        else:
            n_samples = 1
            
        # mask
        if mask == None:
            mask = tensor.alloc(1., state_below.shape[0], 1)
        
        dim = tparams[_p(prefix, 'U')].shape[0]
        
        # initial/previous state
        if init_state == None:
            init_state = tensor.alloc(0., n_samples, dim)
        
        # initial/previous memory
        if init_memory == None:
            init_memory = tensor.alloc(0., n_samples, dim)
            
        # Projected context
        pctx_ = tensor.dot(context, tparams[_p(prefix, 'Wc_att')]) + tparams[_p(prefix, 'b_att')]
        
        if one_step:
            pctx_ = T.addbroadcast(pctx_, 0)
            
        # projected x
        state_below = tensor.dot(state_below, tparams[_p(prefix, 'W')]) + tparams[_p(prefix, 'b')]
        
        Wd_att = tparams[_p(prefix, 'Wd_att')]
        U_att = tparams[_p(prefix, 'U_att')]
        c_att = tparams[_p(prefix), 'c_tt']
        
        if options['selector']:
            W_sel = tparams[_p(prefix, 'W_sel')]
            b_sel = tparams[_p(prefix,'b_sel')]
        else:
            W_sel = T.alloc(0., 1)
            b_sel = T.alloc(0., 1)
        
        U = tparams[_p(prefix, 'U')]
        Wc = tparams[_p(prefix, 'Wc')]
        
        def _slice(_x, n, dim):
            if _x.ndim == 3:
                return _x[:, :, n*dim:(n+1)*dim]
            return _x[:, n*dim:(n+1)*dim]
    
